In [1]:
import pyspark
import os

In [2]:
#only for local use
#in case problem with PYTHONHASHSEED occurs on cluster see https://www.thoughtvector.io/blog/python-3-on-spark-return-of-the-pythonhashseed/
#don't forget to unset on local env afterwards
os.environ["PYTHONHASHSEED"] = "42"

In [3]:
sc = pyspark.SparkContext("local[*]")

In [4]:
def read_graph(path_to_file): 
    res = sc.textFile(path_to_file).map(lambda x : x.split("\t"))
    res = res.flatMap(lambda x : [(x[0], (x[1], int(x[2]))), (x[1], (x[0], int(x[3])))])
    return res

In [27]:
def read_twitter_graph(path_to_file, add_random_weights=False):
    res = sc.textFile(path_to_file).map(lambda x : tuple(x.split(" ")))
    return res

In [28]:
test = read_twitter_graph("twitter_combined.txt")

In [29]:
test.collect()

[('214328887', '34428380'),
 ('17116707', '28465635'),
 ('380580781', '18996905'),
 ('221036078', '153460275'),
 ('107830991', '17868918'),
 ('151338729', '222261763'),
 ('19705747', '34428380'),
 ('222261763', '88323281'),
 ('19933035', '149538028'),
 ('158419434', '17434613'),
 ('149538028', '153226312'),
 ('364971269', '153226312'),
 ('100581193', '279787626'),
 ('113058991', '69592091'),
 ('151338729', '187773078'),
 ('406628822', '262802533'),
 ('460282402', '88323281'),
 ('280935165', '437804658'),
 ('222261763', '27633075'),
 ('285312927', '151338729'),
 ('279787626', '131613362'),
 ('158419434', '17675120'),
 ('394263193', '100581193'),
 ('254839786', '88323281'),
 ('204317520', '21548772'),
 ('67864340', '172883064'),
 ('270449528', '297801196'),
 ('153226312', '187773078'),
 ('67864340', '8088112'),
 ('153226312', '17116707'),
 ('394263193', '14925700'),
 ('124528830', '307458983'),
 ('204317520', '160237722'),
 ('220368467', '54228724'),
 ('206923844', '103598216'),
 ('15322

## graph non pondéré

In [5]:
#very small graph
directions = sc.parallelize([(1,2),(1,3),(2,4),(3,5),(4,5)])
begin = 1
end = 5
shortest_paths = sc.parallelize([(begin, (0,[]))])

In [50]:
directions = read_twitter_graph("twitter_combined.txt")

In [51]:
begin, end = '15519274', '309366491'
shortest_paths = sc.parallelize([(begin, (0,[]))])
early_stop = 15

In [48]:
%%time
i = 0
while True:
    res = shortest_paths.lookup(end)
    i+=1
    print(i)
    if not (res == [] and i < early_stop):
        break

    shortest_paths = shortest_paths.join(directions).map(lambda x : (x[1][1], (x[1][0][0] + 1 , x[1][0][1] + [x[0]])))

1
2
3
4
CPU times: user 140 ms, sys: 50 ms, total: 190 ms
Wall time: 57.5 s


In [55]:
shortest_paths = shortest_paths.join(directions).map(lambda x : (x[1][1], (x[1][0][0] + 1 , x[1][0][1] + [x[0]])))

In [61]:
shortest_paths.groupByKey().map(lambda x : len(x[1])).collect()

[90,
 347,
 234,
 415,
 336,
 2108,
 533,
 6,
 3,
 2,
 11,
 2,
 2,
 3,
 3,
 3,
 2,
 4,
 19,
 23,
 47,
 18,
 11,
 16,
 1,
 4,
 18,
 2,
 1,
 74,
 204,
 340,
 2279,
 27,
 5998,
 233,
 19,
 365,
 881,
 172,
 204,
 1317,
 311,
 180,
 240,
 65,
 467,
 530,
 35,
 577,
 37,
 10,
 214,
 149,
 46,
 9,
 154,
 21,
 227,
 84,
 359,
 464,
 1,
 233,
 4,
 89,
 24,
 115,
 71,
 720,
 94,
 104,
 71,
 278,
 4,
 42,
 4,
 43,
 4,
 6,
 4,
 6284,
 632,
 52,
 12350,
 1068,
 6344,
 66,
 60,
 9243,
 272,
 1979,
 189,
 6469,
 1572,
 5013,
 385,
 413,
 3346,
 1,
 2,
 145,
 104,
 41,
 15,
 50,
 39,
 346,
 6,
 20,
 2,
 127,
 19,
 21,
 138,
 3,
 14,
 15,
 760,
 21,
 27,
 190,
 13,
 78,
 1,
 12,
 105,
 1,
 4,
 18,
 5,
 42,
 18,
 27,
 11,
 184,
 44,
 12,
 18,
 24,
 140,
 22,
 6,
 6,
 6,
 6,
 6,
 80,
 3,
 8,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 81,
 3,
 3,
 1,
 19,
 711,
 442,
 32,
 4,
 78,
 11,
 7,
 3,
 111,
 8,
 3,
 2,
 3,
 32,
 2,
 2,
 2,
 3,
 1,
 6,
 14,
 1,
 231,
 1318,
 642,
 514,
 158,
 66,
 15,
 1,
 5,
 1944,
 1211,

## graph pondéré

#### V1 (brute force)

In [ ]:
#connections are denoted (origin, end, weight)

In [6]:
def compute_path(x):
    #x is the result of the join operation
    # the join should be in format (origin, ((weight_to_origin, path_to_origin, paths_visited_to_origin), (destination, weight_to_destination)))
    return (x[1][1][0], (x[1][0][0] + x[1][1][1], x[1][0][1] + [x[0]], {x[0]}))

In [7]:
def shortest_path_to_point(x,y):
    if x[0] <= y[0]:
        res = (x[0], x[1], x[2] | y[2])
    else:
        res = (y[0], y[1], x[2] | y[2])
    return res

In [6]:
directions = sc.parallelize([(1,(2,1)),(1,(3, 5)),(3,(2,2)),(2,(4,2)),(3,(5,2)),(4,(5,2))])

In [7]:
begin = 1
objective = 5

In [8]:
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).filter(lambda x : x[0] == objective).collect()[0][1])
shortest_paths = sc.parallelize([(begin, (0,[],set()))])

In [9]:
paths_to_objective

{'5'}

In [10]:
%%time
i = 0
while True:
    #stopping criteria
    i += 1
    print(i)
    if i > early_stop :
        break
    path_found = shortest_paths.filter(lambda x : x[0] == objective).collect()
    print(path_found)
    if path_found != [] and path_found[0][1][2] == paths_to_objective :
        break
        
    #real thing
    shortest_paths = shortest_paths.join(directions).map(compute_path).union(shortest_paths)
    shortest_paths = shortest_paths.reduceByKey(shortest_path_to_point)

1
[]
2
[]
3
[]
4
[('6344', (1000, ['226', '7664', '5'], {'5'}))]
CPU times: user 160 ms, sys: 30 ms, total: 190 ms
Wall time: 7.35 s


In [169]:
shortest_paths.collect()

[('3926', (891, ['226', '7664', '5'], {'5'})),
 ('2369', (425, ['226'], {'226', '445'})),
 ('9857', (179, ['226'], {'226'})),
 ('226', (0, [], {'2369', '4542', '7664', '9857'})),
 ('7068', (761, ['226', '7664', '5'], {'5'})),
 ('7664', (168, ['226'], {'226', '5'})),
 ('4542', (210, ['226'], {'226'})),
 ('8920', (418, ['226', '7664', '5'], {'5'})),
 ('236', (1057, ['226', '2369', '445'], {'445'})),
 ('6783', (446, ['226', '7664', '5'], {'5'})),
 ('445', (510, ['226', '2369'], {'236', '2369', '9802'})),
 ('9802', (948, ['226', '2369', '445'], {'445'})),
 ('6344', (1000, ['226', '7664', '5'], {'5'})),
 ('5',
  (367, ['226', '7664'], {'3926', '6344', '6783', '7068', '7664', '8920'}))]

#### V2

In [14]:
def compute_path(x):
    #x is the result of the join operation
    # the join should be in format (origin, ((weight_to_origin, path_to_origin, paths_visited_to_origin), (destination, weight_to_destination)))
    return (x[1][1][0], {
                         "weight_of_path" : x[1][0]["weight_of_path"] + x[1][1][1], 
                         "path" : x[1][0]["path"] + [x[0]], 
                         "explored_path" : {x[0]}
    })

In [13]:
def shortest_path_to_point(x,y):
    if x["weight_of_path"] <= y["weight_of_path"]:
        res = {"weight_of_path" : x["weight_of_path"], 
               "path" : x["path"], 
               "explored_path" : x["explored_path"] | y["explored_path"]}
    else:
        res = {"weight_of_path" : y["weight_of_path"], 
               "path" : y["path"], 
               "explored_path" : x["explored_path"] | y["explored_path"]}
    return res

In [23]:
# for small graph
begin = 1
objective = 5
early_stop = 1
directions = sc.parallelize([(1,(2,1)),(1,(3, 5)),(3,(2,2)),(2,(4,2)),(3,(5,2)),(4,(5,2))])
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).lookup(5))
shortest_paths = sc.parallelize([(begin, {"weight_of_path" :0, "path" : [], "explored_path" : set()})])
points_to_drop = sc.broadcast(set())
continue_criteria = True

In [17]:
# for medium graph but badly connected
directions = read_graph("graph.txt")
begin, objective = "226", "6344" 
early_stop = 20
paths_to_objective = set(directions.map(lambda x : (x[1][0] , x[0])).lookup(5))
shortest_paths = sc.parallelize([(begin, {"weight_of_path" :0, "path" : [], "explored_path" : set()})])
points_to_drop = sc.broadcast(set())
continue_criteria = True

In [18]:
%%time
i = 0
while continue_criteria:
    new_paths = shortest_paths.join(directions).map(compute_path)
    #print(new_paths.collect())
    try:
        min_new_paths =sc.broadcast(new_paths.map(lambda x : x[1]["weight_of_path"]).min())
        points_to_drop = sc.broadcast(set(shortest_paths.filter(lambda x : x[1]["weight_of_path"] < min_new_paths.value).keys().collect()) | points_to_drop.value)
    except ValueError:
        #if no new paths are detected
        min_new_paths = sc.broadcast(float("inf"))
    #print(min_new_paths.value)
    shortest_paths = new_paths.union(shortest_paths).reduceByKey(shortest_path_to_point).filter(lambda x : x[0] not in points_to_drop.value)
    directions = directions.filter(lambda x : x[0] not in points_to_drop.value and x[1][0] not in points_to_drop.value)
    #print(shortest_paths.collect())
    
    #stopping criteria
    continue_criteria = i < early_stop
    i +=1
    print(i)
    try: 
        #replace by lookup on real machine when hash problem is resolved
        path_to_objective = shortest_paths.filter(lambda x : x[0] == objective).collect()
        continue_criteria = not path_to_objective[0][1]["weight_of_path"] <= min_new_paths.value
    except IndexError:
        continue_criteria = min_new_paths.value != float("inf")
        pass
    except KeyError:
        continue_criteria = min_new_paths.value != float("inf")
        pass
        

1
[]
[('4542', {'weight_of_path': 210, 'path': ['226'], 'explored_path': {'226'}}), ('9857', {'weight_of_path': 179, 'path': ['226'], 'explored_path': {'226'}}), ('7664', {'weight_of_path': 168, 'path': ['226'], 'explored_path': {'226'}}), ('2369', {'weight_of_path': 425, 'path': ['226'], 'explored_path': {'226'}})]
168
2
[]
[('5', {'weight_of_path': 367, 'path': ['226', '7664'], 'explored_path': {'7664'}}), ('445', {'weight_of_path': 510, 'path': ['226', '2369'], 'explored_path': {'2369'}})]
367
3
[('6344', {'weight_of_path': 1000, 'path': ['226', '7664', '5'], 'explored_path': {'5'}})]
[('2369', {'weight_of_path': 595, 'path': ['226', '2369', '445'], 'explored_path': {'445'}}), ('236', {'weight_of_path': 1057, 'path': ['226', '2369', '445'], 'explored_path': {'445'}}), ('9802', {'weight_of_path': 948, 'path': ['226', '2369', '445'], 'explored_path': {'445'}}), ('8920', {'weight_of_path': 418, 'path': ['226', '7664', '5'], 'explored_path': {'5'}}), ('6344', {'weight_of_path': 1000, 'p

In [128]:
shortest_paths.filter(lambda x : x[0] == objective).collect()

[('6344',
  {'explored_path': {'5'},
   'path': ['226', '7664', '5'],
   'weight_of_path': 1000})]

SyntaxError: invalid syntax (<ipython-input-145-1533247940e0>, line 1)